<a href="https://colab.research.google.com/github/Maria-lin/How-to-make-notebook-in-dataiku/blob/main/lesfonctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

def repartition_requetes(df, date_x, colonne_date='date', colonne_circuit='circuit'):
    """
    Calcule pour chaque circuit à une date donnée :
    - le nombre de requêtes
    - le total des requêtes pour cette date (nb_requetes_max)
    - le pourcentage de chaque circuit
    """
    # Filtrer les données sur la date demandée
    df_filtre = df[df[colonne_date] == date_x]

    # Si aucune donnée pour cette date, retourner un df vide
    if df_filtre.empty:
        return pd.DataFrame(columns=[colonne_date, colonne_circuit, 'nb_requetes', 'nb_requetes_max', 'pourcentage'])

    # Compter le nombre de requêtes par circuit
    counts = (
        df_filtre.groupby(colonne_circuit)
        .size()
        .reset_index(name='nb_requetes')
        .sort_values(by='nb_requetes', ascending=False)
    )

    # Calcul du total pour la date
    total_requetes = counts['nb_requetes'].sum()

    # Ajouter les colonnes demandées
    counts['nb_requetes_max'] = total_requetes
    counts['pourcentage'] = (counts['nb_requetes'] / total_requetes * 100).round(1)
    counts.insert(0, colonne_date, date_x)

    return counts.reset_index(drop=True)


In [2]:
import pandas as pd

def repartition_flux_par_apiuri(df, en_date, colonne_date='date', colonne_circuit='circuit', colonne_apiuri='apiURI'):
    """
    Calcule pour le circuit RVM et une date donnée :
    - le nombre de flux par apiURI
    - le pourcentage de ces flux par rapport au total RVM de cette date
    - affiche le tout trié par pourcentage décroissant
    """
    # Filtrer sur la date et le circuit RVM
    df_filtre = df[(df[colonne_date] == en_date) & (df[colonne_circuit] == 'RVM')]

    # Si aucun flux trouvé, retourner un df vide
    if df_filtre.empty:
        return pd.DataFrame(columns=[colonne_date, colonne_circuit, colonne_apiuri, 'nb_flux', 'pourcentage'])

    # Remplacer les apiURI manquants par 'NA'
    df_filtre[colonne_apiuri] = df_filtre[colonne_apiuri].fillna('NA')

    # Calcul du nombre de flux par apiURI
    counts = (
        df_filtre.groupby(colonne_apiuri)
        .size()
        .reset_index(name='nb_flux')
        .sort_values(by='nb_flux', ascending=False)
    )

    # Calcul du total
    total_flux = counts['nb_flux'].sum()

    # Ajout du pourcentage
    counts['pourcentage'] = (counts['nb_flux'] / total_flux * 100).round(2)

    # Ajout des colonnes date et circuit
    counts.insert(0, colonne_circuit, 'RVM')
    counts.insert(0, colonne_date, en_date)

    # Tri par pourcentage décroissant
    counts = counts.sort_values(by='pourcentage', ascending=False).reset_index(drop=True)

    return counts


In [1]:
import pandas as pd

def stats_globales(df, end_date, circuit):
    # --- Appel des deux fonctions existantes ---
    df_autres = stats_PSP_stand_alone(df, end_date, circuit)
    df_sepa = stats_SEPAmail(df, end_date, circuit)

    # --- Somme totale des flux pour chaque regroupement ---
    total_autres = df_autres["Nombre"].sum()
    total_sepa = df_sepa["Nombre"].sum()

    # --- Total global ---
    total_general = total_autres + total_sepa

    # --- Construction du tableau final ---
    data = {
        "Regroupement": ["VOP autres RVM (PSP stand-alone)", "VOP SEPAmail"],
        "Nombre de flux": [total_autres, total_sepa],
        "Pourcentage (%)": [
            round((total_autres / total_general) * 100, 2),
            round((total_sepa / total_general) * 100, 2)
        ]
    }

    df_final = pd.DataFrame(data)

    # --- Optionnel : ajouter une ligne de total ---
    df_total = pd.DataFrame({
        "Regroupement": ["Total général"],
        "Nombre de flux": [total_general],
        "Pourcentage (%)": [100.00]
    })

    df_final = pd.concat([df_final, df_total], ignore_index=True)

    return df_final


In [2]:
import pandas as pd

def tableau_general_vop(df, end_date):
    """
    Retourne un tableau général avec :
      - la date
      - le regroupement (VOP SEPAMAIL / VOP autres RVM et PSP stand-alone)
      - le nombre de flux
      - le pourcentage sur le total général

    Args :
      df : DataFrame complet
      end_date : date (datetime.date ou string)
    """
    # S'assurer que la colonne date est bien en datetime
    if not pd.api.types.is_datetime64_any_dtype(df["date"]):
        df["date"] = pd.to_datetime(df["date"], errors='coerce')

    # Filtrer sur la date demandée
    df_date = df[df["date"].dt.date == pd.to_datetime(end_date).date()].copy()

    # Regroupement SEPAmail
    df_sepamail = df_date[
        (df_date["apiUri"] == "https://vop-prod.sepamail.eu/vop/v1/payee-verifications") &
        (df_date["environment"] == "L")
    ]

    # Regroupement Autres RVM / PSP Stand-alone
    df_autres = df_date[
        (df_date["apiUri"] != "https://vop-prod.sepamail.eu/vop/v1/payee-verifications") &
        (df_date["environment"] == "L")
    ]

    # Comptage des flux
    nb_sepamail = len(df_sepamail)
    nb_autres = len(df_autres)
    total = nb_sepamail + nb_autres

    # Construction du tableau final
    data = [
        {
            "date": pd.to_datetime(end_date).date(),
            "Regroupement": "VOP_SEPAMAIL",
            "Nombre de flux": nb_sepamail,
            "Pourcentage (%)": round((nb_sepamail / total) * 100, 2) if total > 0 else 0
        },
        {
            "date": pd.to_datetime(end_date).date(),
            "Regroupement": "VOP_AUTRES_RVM_PSP_STANDALONE",
            "Nombre de flux": nb_autres,
            "Pourcentage (%)": round((nb_autres / total) * 100, 2) if total > 0 else 0
        }
    ]

    df_final = pd.DataFrame(data)
    return df_final
